# Simple usage of a set of MPI engines

This example assumes you've started a cluster of N engines (4 in this example) as part
of an MPI world.  

Our documentation describes [how to create an MPI profile](https://ipyparallel.readthedocs.io/en/stable/process.html#using-ipcluster-in-mpiexec-mpirun-mode)
and explains [basic MPI usage of the IPython cluster](https://ipyparallel.readthedocs.io/en/stable/mpi.html).


For the simplest possible way to start 4 engines that belong to the same MPI world, 
you can run this in a terminal:

<pre>
ipcluster start --engines=MPI -n 4
</pre>

or start an MPI cluster from the cluster tab if you have one configured.

Once the cluster is running, we can connect to it and open a view into it:

In [1]:
import ipyparallel as ipp
rc = ipp.Client()
view = rc[:]

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json


KeyboardInterrupt: 

Let's define a simple function that gets the MPI rank from each engine.

In [7]:
@view.remote(block=True)
def mpi_rank():
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    return comm.Get_rank()

In [8]:
mpi_rank()

[0, 0, 0, 0]

To get a mapping of IPython IDs and MPI rank (these do not always match),
you can use the get_dict method on AsyncResults.

In [9]:
mpi_rank.block = False
ar = mpi_rank()
ar.get_dict()

{0: 0, 1: 0, 2: 0, 3: 0}

With %%px cell magic, the next cell will actually execute *entirely on each engine*:

In [10]:
%%px
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
   data = [(i+1)**2 for i in range(size)]
else:
   data = None
data = comm.scatter(data, root=0)

assert data == (rank+1)**2, 'data=%s, rank=%s' % (data, rank)
{
    'data': data,
    'rank': rank,
}

{'data': 1, 'rank': 0}

{'data': 1, 'rank': 0}

{'data': 1, 'rank': 0}

{'data': 1, 'rank': 0}